# 📝 Exercise M3.01

The goal is to write an exhaustive search to find the best parameters
combination maximizing the model generalization performance.

Here we use a small subset of the Adult Census dataset to make the code
faster to execute. Once your code works on the small subset, try to
change `train_size` to a larger value (e.g. 0.8 for 80% instead of
20%).

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

adult_census = pd.read_csv("../datasets/adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.2, random_state=42)

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)
preprocessor = ColumnTransformer(
    [('cat_preprocessor', categorical_preprocessor,
      selector(dtype_include=object))],
    remainder='passthrough', sparse_threshold=0)

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=42))
])


Use the previously defined model (called `model`) and using two nested `for`
loops, make a search of the best combinations of the `learning_rate` and
`max_leaf_nodes` parameters. In this regard, you will need to train and test
the model by setting the parameters. The evaluation of the model should be
performed using `cross_val_score` on the training set. We will use the
following parameters search:
- `learning_rate` for the values 0.01, 0.1, 1 and 10. This parameter controls
  the ability of a new tree to correct the error of the previous sequence of
  trees
- `max_leaf_nodes` for the values 3, 10, 30. This parameter controls the
  depth of each tree.

In [3]:
model.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                     transformers=[('cat_preprocessor',
                                    OrdinalEncoder(handle_unknown='use_encoded_value',
                                                   unknown_value=-1),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x7fb6dc8e38b0>)])),
  ('classifier', HistGradientBoostingClassifier(random_state=42))],
 'verbose': False,
 'preprocessor': ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                   transformers=[('cat_preprocessor',
                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                 unknown_value=-1),
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb6dc8e38b0>)]),
 'classifier': HistGradientBoostingClassifier(random_st

In [11]:
# Write your code here.
from sklearn.model_selection import cross_val_score
import numpy as np


lr = np.logspace(-2,1,num=4)
max_leaf_nodes = [3, 10, 30]

for each_lr in lr:
    model.set_params(classifier__learning_rate=each_lr)
    for each_max_leaf_node in max_leaf_nodes:
        model.set_params(classifier__max_leaf_nodes=each_max_leaf_node)
        cv_results = cross_val_score(model, data_train, target_train, error_score='raise')
        print(f"For lr={each_lr} and max_leaf_nodes={each_max_leaf_node}"
        f" the mean cross validated score for the training set is:{cv_results.mean():.3f}" 
              f" +/- {cv_results.std():.3f}")

For lr=0.01 and max_leaf_nodes=3 the mean cross validated score for the training set is:0.790 +/- 0.004
For lr=0.01 and max_leaf_nodes=10 the mean cross validated score for the training set is:0.814 +/- 0.002
For lr=0.01 and max_leaf_nodes=30 the mean cross validated score for the training set is:0.842 +/- 0.007
For lr=0.1 and max_leaf_nodes=3 the mean cross validated score for the training set is:0.849 +/- 0.004
For lr=0.1 and max_leaf_nodes=10 the mean cross validated score for the training set is:0.863 +/- 0.005
For lr=0.1 and max_leaf_nodes=30 the mean cross validated score for the training set is:0.861 +/- 0.006
For lr=1.0 and max_leaf_nodes=3 the mean cross validated score for the training set is:0.854 +/- 0.006
For lr=1.0 and max_leaf_nodes=10 the mean cross validated score for the training set is:0.837 +/- 0.008
For lr=1.0 and max_leaf_nodes=30 the mean cross validated score for the training set is:0.819 +/- 0.013
For lr=10.0 and max_leaf_nodes=3 the mean cross validated score 

Better Implementation the earlier one

In [ ]:
from sklearn.model_selection import cross_val_score

learning_rate = [0.01, 0.1, 1, 10]
max_leaf_nodes = [3, 10, 30]

best_score = 0
best_params = {}
for lr in learning_rate:
    for mln in max_leaf_nodes:
        print(f"Evaluating model with learning rate {lr:.3f}"
              f" and max leaf nodes {mln}... ", end="")
        model.set_params(
            classifier__learning_rate=lr,
            classifier__max_leaf_nodes=mln
        )
        scores = cross_val_score(model, data_train, target_train, cv=2)
        mean_score = scores.mean()
        print(f"score: {mean_score:.3f}")
        if mean_score > best_score:
            best_score = mean_score
            best_params = {'learning-rate': lr, 'max leaf nodes': mln}
            print(f"Found new best model with score {best_score:.3f}!")

print(f"The best accuracy obtained is {best_score:.3f}")
print(f"The best parameters found are:\n {best_params}")


Now use the test set to score the model using the best parameters
that we found using cross-validation in the training set.

In [6]:
# Write your code here.
model.set_params(classifier__learning_rate=0.1,
                classifier__max_leaf_nodes=30)
cv_results= cross_val_score(model, data_test, target_test)
print(f"The test set score for lr=0.1 and max_leaf_nodes=30 is {cv_results.mean():.3f} +/- {cv_results.std():.3f}")

The test set score for lr=0.1 and max_leaf_nodes=30 is 0.875 +/- 0.004


Better Implementation the earlier one

In [ ]:
best_lr = best_params['learning-rate']
best_mln = best_params['max leaf nodes']

model.set_params(classifier__learning_rate=best_lr,
                 classifier__max_leaf_nodes=best_mln)
model.fit(data_train, target_train)
test_score = model.score(data_test, target_test)

print(f"Test score after the parameter tuning: {test_score:.3f}")